In [1]:
from omop_alchemy.helpers.populate_db import remove_foreign_keys, get_foreign_keys_from_ORM, reinstate_foreign_keys

omop_alchemy.db.config - DEBUG - Application config path set: /Users/georginakennedy/cloudstor/CBDRH/ACDN/OMOP_Alchemy/oa_system_config.yaml
omop_alchemy.db.config - DEBUG - Log path set: /Users/georginakennedy/cloudstor/CBDRH/ACDN/OMOP_Alchemy/omop_alchemy/logs
omop_alchemy.db.config - DEBUG - DB connection string: sqlite:////Users/georginakennedy/cloudstor/CBDRH/ACDN/OMOP_Alchemy/omop_alchemy/resources/dash.db


In [2]:
import omop_alchemy as oa
from omop_alchemy import oa_config, Base

In [3]:
oa_config.engine

Engine(sqlite:////Users/georginakennedy/cloudstor/CBDRH/ACDN/OMOP_Alchemy/omop_alchemy/resources/dash.db)

In [4]:
remove_foreign_keys(oa_config.engine, Base.metadata.tables)


omop_alchemy.db.config - ERROR - (sqlite3.OperationalError) near "CONSTRAINT": syntax error
[SQL: ALTER TABLE concept_ancestor DROP CONSTRAINT ca_fk_1]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
omop_alchemy.db.config - ERROR - (sqlite3.OperationalError) near "CONSTRAINT": syntax error
[SQL: ALTER TABLE concept_ancestor DROP CONSTRAINT ca_fk_2]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
omop_alchemy.db.config - ERROR - (sqlite3.OperationalError) near "CONSTRAINT": syntax error
[SQL: ALTER TABLE concept_class DROP CONSTRAINT cc_fk_1]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
omop_alchemy.db.config - ERROR - (sqlite3.OperationalError) near "CONSTRAINT": syntax error
[SQL: ALTER TABLE concept_relationship DROP CONSTRAINT cr_fk_1]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
omop_alchemy.db.config - ERROR - (sqlite3.OperationalError) near "CONSTRAINT": syntax error
[SQL: ALTER TABLE concept_relationship DROP CONSTRAINT 

[ForeignKeyConstraint(<sqlalchemy.sql.base.ReadOnlyColumnCollection object at 0x1183e9df0>, None, name='ca_fk_1', table=Table('concept_ancestor', MetaData(), schema=None)),
 ForeignKeyConstraint(<sqlalchemy.sql.base.ReadOnlyColumnCollection object at 0x1183644a0>, None, name='ca_fk_2', table=Table('concept_ancestor', MetaData(), schema=None)),
 ForeignKeyConstraint(<sqlalchemy.sql.base.ReadOnlyColumnCollection object at 0x118366430>, None, name='cc_fk_1', table=Table('concept_class', MetaData(), schema=None)),
 ForeignKeyConstraint(<sqlalchemy.sql.base.ReadOnlyColumnCollection object at 0x1182a4c70>, None, name='cr_fk_1', table=Table('concept_relationship', MetaData(), schema=None)),
 ForeignKeyConstraint(<sqlalchemy.sql.base.ReadOnlyColumnCollection object at 0x11820b790>, None, name='cr_fk_2', table=Table('concept_relationship', MetaData(), schema=None)),
 ForeignKeyConstraint(<sqlalchemy.sql.base.ReadOnlyColumnCollection object at 0x1182090d0>, None, name='cr_fk_3', table=Table('con

In [ ]:

from sqlalchemy.engine import reflection
from sqlalchemy.ext.declarative import DeclarativeMeta
import inspect
from sqlalchemy.orm.attributes import InstrumentedAttribute
from sqlalchemy.schema import DropConstraint, AddConstraint
import sqlalchemy as sa
from omop_alchemy import logger

In [ ]:

inspector = reflection.Inspector.from_engine(oa_config.engine)
fake_metadata = sa.MetaData()

fake_tables = []
all_fks = []

for table_name in Base.metadata.tables:
    print(table_name)
    if 'temp' not in table_name:
        try:
            fks = []
            for i, fk in enumerate(inspector.get_foreign_keys(table_name)):
                print(fk)
                #if fk['name']:
                fks.append(sa.ForeignKeyConstraint((),(),name=f'fk_{i}'))#fk['name']))
            t = sa.Table(table_name, fake_metadata, *fks)
            fake_tables.append(t)
            all_fks.extend(fks)
        except:
            if table_name != 'Base':
                logger.debug(f'error removing fk from table {table_name}')
connection = oa_config.engine.connect()
transaction = connection.begin()
for fkc in all_fks:
    try:
        connection.execute(DropConstraint(fkc))
    except Exception as e:
        logger.error(e)
transaction.commit()
connection.close()

In [ ]:
all_fks

In [ ]:
import sqlalchemy.orm as so

In [ ]:
with so.Session(oa_config.engine) as session:
    reinstate_foreign_keys(session, oa_config.engine, oa.tables.clinical.__all__, Base)

In [ ]:

def reinstate_foreign_keys(sess, engine, Model, Base, errors_to_script=False):
    remove_foreign_keys(engine, Base.metadata.tables)
    all_fks = get_foreign_keys_from_ORM(Model)
    connection = engine.connect()
    error_script = []
    complete = 0
    for fkc in all_fks:
        transaction = connection.begin()
        tab, col = str(fkc.columns[fkc.column_keys[0]]).split('.')
        ref_tab, ref_col = str(fkc.elements[0].column).split('.')
        try:
            logger.debug(f'adding constraint: {tab}.{col} --> {ref_tab}.{ref_col}')
            connection.execute(AddConstraint(fkc))
            logger.debug(f'constraint added: {tab}.{col} --> {ref_tab}.{ref_col}')
            complete += 1
        except Exception as e:
            # we only have support for simple foreign key relationships
            # at the moment - assumptions being made that there is only
            # one column_key and only one referred element
            # sess.rollback()
            logger.error(e, 'error')
            logger.warning(f'failed to add constraint: {tab}.{col} --> {ref_tab}.{ref_col}')
            error_script.append(f'select distinct a1.{col} from {tab} as a1 left outer join {ref_tab} as '\
                                f'a2 on a1.{col} = a2.{ref_col} where a2.{ref_col} is null;')
        transaction.commit()
    logger.info(f'successfully added {complete} of {len(all_fks)} constraints')
    if errors_to_script:
        for e in error_script:
            print(e)